In [1]:
import pandas as pd

frame = pd.DataFrame({
    'color':  ['white','red','green','red','green'],
    'object': ['pen','pencil','pencil','ashtray','pen'],
    'price1': [5.56, 4.20, 1.30, 0.56, 2.75],
    'price2': [4.75, 4.12, 1.60, 0.75, 3.15]
})
frame


,color,object,price1,price2
0,white,pen,5.56,4.75
1,red,pencil,4.20,4.12
2,green,pencil,1.30,1.60
3,red,ashtray,0.56,0.75
4,green,pen,2.75,3.15


In [2]:
group = frame['price1'].groupby(frame['color'])
group          # об'єкт типу SeriesGroupBy

In [3]:
group.groups


{'green': [2, 4], 'red': [1, 3], 'white': [0]}

In [4]:
group.mean()

color
green    2.025
red      2.380
white    5.560
Name: price1, dtype: float64

In [5]:
group.sum()

color
green    4.05
red      4.76
white    5.56
Name: price1, dtype: float64

In [6]:
frame = pd.DataFrame({
    'color':  ['white','red','green','red','green', 'red'],
    'object': ['pen','pencil','pencil','ashtray','pen', 'pencil'],
    'price1': [5.56, 4.20, 1.30, 0.56, 2.75, 4.20],
    'price2': [4.75, 4.12, 1.60, 0.75, 3.15, 4.12]
})
frame

,color,object,price1,price2
0,white,pen,5.56,4.75
1,red,pencil,4.20,4.12
2,green,pencil,1.30,1.60
3,red,ashtray,0.56,0.75
4,green,pen,2.75,3.15
5,red,pencil,4.20,4.12


In [7]:
ggroup = frame['price1'].groupby([frame['color'], frame['object']])
ggroup.groups

{('green', 'pen'): [4], ('green', 'pencil'): [2], ('red', 'ashtray'): [3], ('red', 'pencil'): [1, 5], ('white', 'pen'): [0]}

In [8]:
ggroup.sum()           # сума price1 для кожної пари ключів

color  object 
green  pen        2.75
       pencil     1.30
red    ashtray    0.56
       pencil     8.40
white  pen        5.56
Name: price1, dtype: float64

In [9]:
frame[['price1','price2']].groupby(frame['color']).mean()

,price1,price2
color,,
green,2.025000,2.375000
red,2.986667,2.996667
white,5.560000,4.750000


In [10]:
frame.groupby(frame['color']).mean(numeric_only=True)

,price1,price2
color,,
green,2.025000,2.375000
red,2.986667,2.996667
white,5.560000,4.750000


In [11]:
frame

,color,object,price1,price2
0,white,pen,5.56,4.75
1,red,pencil,4.20,4.12
2,green,pencil,1.30,1.60
3,red,ashtray,0.56,0.75
4,green,pen,2.75,3.15
5,red,pencil,4.20,4.12


In [12]:
for name, grp in frame.groupby('color'):
    print(name)
    print(grp)

green
   color  object  price1  price2
2  green  pencil    1.30    1.60
4  green     pen    2.75    3.15
red
  color   object  price1  price2
1   red   pencil    4.20    4.12
3   red  ashtray    0.56    0.75
5   red   pencil    4.20    4.12
white
   color object  price1  price2
0  white    pen    5.56    4.75


In [13]:
# 1) вибір колонки ДО groupby
frame['price1'].groupby(frame['color']).mean()

color
green    2.025000
red      2.986667
white    5.560000
Name: price1, dtype: float64

In [14]:
# 2) вибір колонки ПІСЛЯ groupby
frame.groupby(frame['color'])['price1'].mean()


color
green    2.025000
red      2.986667
white    5.560000
Name: price1, dtype: float64

In [15]:
# 3) вибір колонки ПІСЛЯ агрегування числових
frame.groupby(frame['color']).mean(numeric_only=True)['price1']

color
green    2.025000
red      2.986667
white    5.560000
Name: price1, dtype: float64

In [16]:
means = frame.groupby('color').mean(numeric_only=True).add_prefix('mean_')
means
# колонки: mean_price1, mean_price2


,mean_price1,mean_price2
color,,
green,2.025000,2.375000
red,2.986667,2.996667
white,5.560000,4.750000


In [17]:
group = frame.groupby('color')
q60 = group['price1'].quantile(0.6)
q60

color
green    2.17
red      4.20
white    5.56
Name: price1, dtype: float64

In [18]:
def value_range(series):
    return series.max() - series.min()

rng = group['price1'].agg(value_range)
rng

color
green    1.45
red      3.64
white    0.00
Name: price1, dtype: float64

In [19]:
summary = group['price1'].agg(['mean', 'std', value_range])
summary

,mean,std,value_range
color,,,
green,2.025000,1.025305,1.45
red,2.986667,2.101555,3.64
white,5.560000,NaN,0.00


In [20]:
frame = pd.DataFrame({
    'color':  ['white','red','green','red','green'],
    'price1': [5.56, 4.20, 1.30, 0.56, 2.75],
    'price2': [4.75, 4.12, 1.60, 0.75, 3.15]
})

sums = frame.groupby('color').sum().add_prefix('tot_')
merged = pd.merge(frame, sums, left_on='color', right_index=True)
merged

,color,price1,price2,tot_price1,tot_price2
0,white,5.56,4.75,5.56,4.75
1,red,4.20,4.12,4.76,4.87
2,green,1.30,1.60,4.05,4.75
3,red,0.56,0.75,4.76,4.87
4,green,2.75,3.15,4.05,4.75


In [23]:
import numpy as np
totals = frame.groupby('color').transform("sum").add_prefix('tot_')
frame_with_totals = pd.concat([frame, totals], axis=1)
frame_with_totals

,color,price1,price2,tot_price1,tot_price2
0,white,5.56,4.75,5.56,4.75
1,red,4.20,4.12,4.76,4.87
2,green,1.30,1.60,4.05,4.75
3,red,0.56,0.75,4.76,4.87
4,green,2.75,3.15,4.05,4.75


In [28]:
frame = pd.DataFrame({
    'color':  ['white','black','white','white','black','black'],
    'status': ['up','up','down','down','down','up'],
    'value1': [12.33,14.55,22.34,27.84,23.40,18.33],
    'value2': [11.23,31.80,29.99,31.18,18.25,22.44]
})

out = frame.groupby(['color','status']).apply(lambda x: x.max())[['value1','value2']]
print(out)


              value1  value2
color status                
black down     23.40   18.25
      up       18.33   31.80
white down     27.84   31.18
      up       12.33   11.23


/var/folders/5z/_4ymrqb55gl_n9nt577w6kkm0000gn/T/ipykernel_5452/931168853.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  out = frame.groupby(['color','status']).apply(lambda x: x.max())[['value1','value2']]
